In [3]:
try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle
from kaveh.behavioral import oculomotor
from kaveh.sorting import spikesorter
from kaveh.toolbox import find_file
from kaveh.plots import axvlines
import numpy as np
from matplotlib import pyplot as plt
from smr import File
import csv
import os

In [4]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [5]:
f_name = '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2008_Random/Kimo/K48/error_direction/K48_1_CSddirTuning.smr' # good
# f_name = '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2008_Random/Kimo/K45/error_direction/K45_1_ErrorDirectionTest.smr'


In [6]:
smr_content = File(f_name)
smr_content.read_channels()

Not implemented (type = 6)
Not implemented (type = 5)


In [7]:
voltage_chan = smr_content.get_channel(0)


In [8]:

cs_path = '/mnt/data/temp/kaveh/auto_processed/' 
cf = find_file(os.path.split(f_name)[1] + '.pkl', cs_path)
with open(cf, 'rb') as input:
    sss = pickle.load(input)

In [9]:
from neo.io import Spike2IO

neo_reader = Spike2IO(filename=f_name)
neo_data = neo_reader.read()
data_block = neo_data[0]
seg = data_block.segments[0]
# Assuming cs spike trains are saved in the 0th index of spike train channels
cs_labeled = np.array(seg.spiketrains[2])
cs_labeled = np.int32(cs_labeled/sss.dt)

In [10]:
sss.voltage = voltage_chan.data


spike_indices = sss.get_spike_indices()


In [11]:
from kaveh.toolbox import closest_argmin

cs = closest_argmin(cs_labeled, spike_indices)


In [12]:
labeled = np.zeros(spike_indices.shape)
labeled[cs] = 1.0

In [13]:
# from kaveh.plots import axvlines

# plt.close('all')
# plt.figure(figsize=(20,5))

# # plt.plot(sss.voltage)
# # axvlines(plt.gca(), cs_labeled, color = 'r', alpha = 0.2)
# plt.plot(labeled, '.')


In [14]:
# powers = powers[0:cs[-1], :]
labeled = labeled[0:cs[-1]]

In [15]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
aligned_spikes = sss.aligned_spikes[0:cs[-1],:]

pca = PCA(n_components=20)
pca.fit(aligned_spikes )

time_pca = pca.transform(aligned_spikes)
time_pca = normalize(time_pca)

In [16]:
[max_powers,powers,xf] = sss._find_max_powers()
powers = powers[0:cs[-1], :]

pca = PCA(n_components=10)
pca.fit(powers )

freq_pca = pca.transform(powers)
freq_pca = normalize(freq_pca)


In [17]:
X = np.concatenate((freq_pca, time_pca), 1)

In [104]:
X = np.concatenate((X,X_test), 0)
labeled = np.concatenate((labeled, label_test), 0)

In [105]:
X.shape

(59370, 30)

In [64]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', class_weight='balanced').fit(X, labeled)



In [65]:
prediction = clf.predict(X)
print(prediction.shape)
print('Error rate: ', np.count_nonzero((prediction - labeled)==-1)/np.sum(labeled))


(20165,)
('Error rate: ', 0.005543237250554324)


In [20]:
# import sklearn

# print('The scikit-learn version is {}.'.format(sklearn.__version__))


In [21]:
# xvalidation
from sklearn.model_selection import cross_validate

clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
cv_results = cross_validate(clf, X, labeled, cv=3, return_train_score=True, return_estimator = True)

print(cv_results.keys())

print('Test accuracy', cv_results['test_score'])
print('Train accuracy', cv_results['train_score'])



['score_time', 'test_score', 'train_score', 'fit_time', 'estimator']
('Test accuracy', array([0.99717346, 0.99776852, 0.99776819]))
('Train accuracy', array([0.99836346, 0.99784274, 0.99821482]))


In [22]:
f_name2 = '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2008_Random/Kimo/K45/error_direction/K45_1_ErrorDirectionTest.smr'


In [23]:

cs_path = '/mnt/data/temp/kaveh/auto_processed/' 
cf = find_file(os.path.split(f_name2)[1] + '.pkl', cs_path)
with open(cf, 'rb') as input:
    sss = pickle.load(input)

In [24]:

neo_reader = Spike2IO(filename=f_name2)
neo_data = neo_reader.read()
data_block = neo_data[0]
seg = data_block.segments[0]
# Assuming cs spike trains are saved in the 0th index of spike train channels
cs_label_test = np.array(seg.spiketrains[1])
cs_label_test = np.int32(cs_label_test/sss.dt)

In [25]:
smr_content = File(f_name2)
smr_content.read_channels()
voltage_chan = smr_content.get_channel(0)
sss.voltage = voltage_chan.data
spike_indices = sss.get_spike_indices()
cs = closest_argmin(cs_label_test, spike_indices)
label_test = np.zeros(spike_indices.shape)
label_test[cs] = 1.0

Not implemented (type = 6)
Not implemented (type = 5)


In [26]:
# aligned_spikes = sss.aligned_spikes[0:cs[-1],:]

pca = PCA(n_components=20)
pca.fit(sss.aligned_spikes )

time_pca_test = pca.transform(sss.aligned_spikes)
time_pca_test = normalize(time_pca_test)

In [27]:
[max_powers,powers,xf] = sss._find_max_powers()
# powers = powers[0:cs[-1], :]

pca = PCA(n_components=10)
pca.fit(powers )

freq_pca_test = pca.transform(powers)
freq_pca_test = normalize(freq_pca_test)


In [68]:
X_test = np.concatenate((freq_pca_test, time_pca_test), 1)
# clf = cv_results['estimator'][2]
prediction = clf.predict(X_test)
print(prediction.shape)
print('Error rate: ', np.count_nonzero((prediction - label_test) == -1)/np.float(np.sum(label_test)))


(34512,)
('Error rate: ', 0.06918238993710692)


371

In [ ]:
np.size

In [32]:
clf.score(X_test, label_test)

0.9892501159017153